In [ ]:
from ultralytics import YOLO
import os
import shutil
import torch
from datetime import datetime

# Default hyperparameters
config = {
    "epochs": 8,
    "batch_size": 8,
    "mosaic": 0.3,
    "mixup": 0.1,
    "optimizer": "Adam",
    "momentum": 0.8,
    "lr0": 0.001,   # start LR
    "lrf": 0.01,    # final LR fraction
    "single_cls": False,
    "amp": True if torch.cuda.is_available() else False,
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    "weight_decay": 0.0005,
}




In [16]:
def train_model(cfg, save_dir="results"):
    """Train YOLOv8 model with given config dictionary and save results"""
    this_dir = os.getcwd()

    # Load YOLOv8 model
    model = YOLO(os.path.join(this_dir, "yolov8n.pt"))

    print(f"🚀 Training on {cfg['device']} for {cfg['epochs']} epochs "
          f"with batch size {cfg['batch_size']}")

    # Train
    results = model.train(
        data=os.path.join(this_dir, "yolo_params.yaml"),
        epochs=cfg["epochs"],
        batch=cfg["batch_size"],
        device=cfg["device"],
        single_cls=cfg["single_cls"],
        mosaic=cfg["mosaic"],
        mixup=cfg["mixup"],
        optimizer=cfg["optimizer"],
        lr0=cfg["lr0"],
        lrf=cfg["lrf"],
        momentum=cfg["momentum"],
        weight_decay=cfg["weight_decay"],
        val=True,
        amp=cfg["amp"],
    )

    # Ultralytics auto-saves to runs/detect/trainX/
    run_dir = results.save_dir

    # Create timestamped results folder
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    final_save_dir = os.path.join(this_dir, f"{save_dir}_{timestamp}")
    shutil.copytree(run_dir, final_save_dir)

    print(f"✅ Training complete. Results saved in: {final_save_dir}")
    print(f"👉 Best model: {os.path.join(final_save_dir, 'weights', 'best.pt')}")
    return results, final_save_dir

In [17]:
def run_inference(model_path, source="test.jpg", save=True):
    """Run inference using the trained best model"""
    model = YOLO(model_path)
    print(f"🔎 Running inference on: {source}")

    results = model.predict(
        source=source,       # image file or folder
        save=save,           # save output images with boxes
        conf=0.25,           # confidence threshold
        device="cuda" if torch.cuda.is_available() else "cpu"
    )

    if save:
        print(f"✅ Predictions saved in: {results[0].save_dir}")
    return results


In [18]:

# ---- Run everything ----
results, save_path = train_model(config)

# Path to best model
best_model_path = os.path.join(save_path, "weights", "best.pt")

🚀 Training on cpu for 8 epochs with batch size 8
New https://pypi.org/project/ultralytics/8.3.203 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.202  Python-3.12.3 torch-2.6.0+cpu CPU (12th Gen Intel Core i5-1235U)
engine\trainer: agnostic_nms=False, amp=False, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=c:\Users\prane\OneDrive\Documents\GitHub\Astra\Dataset\Hackathon2_scripts\yolo_params.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=8, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.1, mode=t

In [25]:
inference_results = run_inference(best_model_path, source="C:/Users/prane/OneDrive/Documents/GitHub/Astra/Dataset/test1/images/000000031_dark_clutter.png")

🔎 Running inference on: C:/Users/prane/OneDrive/Documents/GitHub/Astra/Dataset/test1/images/000000031_dark_clutter.png

image 1/1 C:\Users\prane\OneDrive\Documents\GitHub\Astra\Dataset\test1\images\000000031_dark_clutter.png: 384x640 1 OxygenTank, 1 NitrogenTank, 2 SafetySwitchPanels, 88.2ms
Speed: 3.9ms preprocess, 88.2ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)
Results saved to C:\Users\prane\runs\detect\predict4
✅ Predictions saved in: C:\Users\prane\runs\detect\predict4
